In [64]:
import sys, random #segmentqueery
import pygame
from pygame.locals import *
import pymunk
import pymunk.pygame_util
from math import sqrt
from math import atan2
from math import degrees
import pandas as pd


class distance_sensor(object):
    def __init__(self,start_x,start_y,end_x,end_y ,space):
        self.start_x = start_x
        self.start_y = start_y
        self.end_x=end_x
        self.end_y=end_y
        self.space= space
    
    def place(self):
        sensor_shape=pymunk.Circle(None,5)
        sensor=pymunk.Body(body_type=pymunk.Body.KINEMATIC)
        sensor_shape.body=sensor
        sensor.position=(self.start_x,self.start_y)
        sensor_shape.filter=pymunk.ShapeFilter(categories=0b1)
        self.space.add(sensor_shape,sensor)

    def distance(self):
        segment_query=self.space.segment_query_first((self.start_x,self.start_y),(self.end_x,self.end_y),1,pymunk.ShapeFilter(mask=pymunk.ShapeFilter.ALL_MASKS ^ 0b1))
        distance = None
        if segment_query is not None:
            distance = segment_query.alpha*sqrt((self.end_x-self.start_x)*(self.end_x-self.start_x)+(self.end_y-self.start_y)*(self.end_y-self.start_y))
        return distance
    
    def get_y_self(self) :
        return self.start_y
     
        
def add_piece(space):
    mass = 100
    radius = 10
    inertia = pymunk.moment_for_segment(mass, (-130,0),(180.0,0.0), radius)
    body = pymunk.Body(mass, inertia)
    x = random.randint(120,380)
    body.position = x, 550
    shape = pymunk.Segment(body ,(-130,0),(190.0,0.0), radius)
    shape.friction=99999
    shape.collision_type = 2
    space.add(body, shape)
    return shape

def add_workspace(space):
    
    body = pymunk.Body(body_type=pymunk.Body.STATIC)
    body.position = (150,150)
    l1 = pymunk.Segment(body, (-150, 0), (500.0, 0.0), 5.0)
    #l2 = pymunk.Segment(body, (-150.0, 20), (-50.0, 20.0), 15) segments sind trash , radius problem
    l1.friction=999999.0
    l1.filter=pymunk.ShapeFilter(categories=0b1)
    
    poly_shape_1=pymunk.Poly.create_box(None,(200.0,30.0),0)
    box_1=pymunk.Body(body_type=pymunk.Body.STATIC)
    poly_shape_1.body=box_1
    box_1.position=90,170
    poly_shape_1.friction=99999999.0
    poly_shape_1.filter=pymunk.ShapeFilter(categories=0b1)
    
    #noch 2 boxen
    poly_shape_2=pymunk.Poly.create_box(None,(48.5,30.0),0)
    box_2=pymunk.Body(body_type=pymunk.Body.STATIC)
    poly_shape_2.body=box_2
    box_2.position=17,200
    poly_shape_2.filter=pymunk.ShapeFilter(categories=0b1)
    
    poly_shape_3=pymunk.Poly.create_box(None ,(97.5,60.0,0))
    box_3=pymunk.Body(body_type=pymunk.Body.STATIC)
    poly_shape_3.body=box_3
    box_3.position=550,185
    poly_shape_3.filter=pymunk.ShapeFilter(categories=0b1)

    #sensoren simu
#     sensor_1_shape=pymunk.Circle(None,5)
#     sensor_1=pymunk.Body(body_type=pymunk.Body.KINEMATIC)
#     sensor_1_shape.body=sensor_1
#     sensor_1.position=501,200
#     sensor_1_shape.filter=pymunk.ShapeFilter(categories=0b1)
    #sensor_1_shape.
    
#     sensor_2_shape=pymunk.Circle(None,5)
#     sensor_2=pymunk.Body(body_type=pymunk.Body.KINEMATIC)
#     sensor_2_shape.body=sensor_2
#     sensor_2.position=43,200
#     sensor_2_shape.filter=pymunk.ShapeFilter(categories=0b1)
    
    
    space.add(l1,box_1,poly_shape_1,box_2,poly_shape_2,box_3,poly_shape_3)
              #sensor_1_shape,sensor_1,sensor_2_shape,sensor_2)
    
    return l1,box_1


# def distance(start_x,start_y,end_x,end_y ,space,shape):
    
    
#     sensor_shape=pymunk.Circle(None,5)
#     sensor=pymunk.Body(body_type=pymunk.Body.KINEMATIC)
#     sensor_shape.body=sensor
#     sensor.position=(start_x,start_y)
#     sensor_shape.filter=pymunk.ShapeFilter(categories=0b1)
#     space.add(sensor_shape,sensor)
    
#     segment_query=space.segment_query_first((start_x,start_y),(end_x,end_y),1,pymunk.ShapeFilter(mask=pymunk.ShapeFilter.ALL_MASKS ^ 0b1))
#     distance = 0 
#     if segment_query is not None:
#         distance = segment_query.alpha*sqrt((end_x-start_x)*(end_x-start_x)+(end_y-start_y)*(end_y-start_y))
#     return distance


def angle (x , y ):
    angle=atan2(y,x)
    return  angle
        
def traitement_case_1 (sensor_left , sensor_right):
    boutton='You cant press'
    press=False
    
    s_right_y_pos=15
    s_left_y_pos=15
    
    s_left_touch=sensor_left.distance()
    s_right=sensor_right.distance()
    
    if s_right==None : s_right=0
        
    if s_left_touch==None : s_left=0
    else : s_left=s_left_touch
    
    winkel_1 = degrees(angle(s_right,s_right_y_pos))
    winkel_2 = degrees(angle(s_left,s_left_y_pos))
    winkel=None
   
    
    if s_left_touch==None and s_right>10:
        boutton='Female'
        press=True
    if s_left<5 and  11.5<=winkel_1<=13.7 :
        winkel=round(winkel_1,3)
        boutton='Male'
        press=True
    if s_right>s_left and 10.3<=winkel_2<10.82 :
        winkel=round(winkel_2,3)
        boutton='Male'
        press=True 
    
    
    return press , boutton ,winkel    
    
   
def main():
    
    i=0
    pygame.init()
    screen = pygame.display.set_mode((600, 600))
    pygame.display.set_caption("wazzup")
    clock = pygame.time.Clock()
    space = pymunk.Space()
    space.gravity = (0.0, -900.0)

    sensor_1_measures=[]
    sensor_2_measures=[]
    
    
    lines = add_workspace(space)
    pieces = []
    draw_options = pymunk.pygame_util.DrawOptions(screen)
    ticks_to_next_piece = 30
    sensor_right=distance_sensor(501,200,0,200,space)
    sensor_left=distance_sensor(190,170,600,200,space)
    sensor_right.place()
    sensor_left.place()
    
    while True:
        
        for event in pygame.event.get():
            if event.type == QUIT:
                sys.exit(0)
            elif event.type == KEYDOWN and event.key == K_ESCAPE:
                sys.exit(0)
        #print (ticks_to_next_piece)
        
        a1=sensor_right.distance()
        a2=sensor_left.distance()
        
        
        sensor_data =traitement_case_1(sensor_left,sensor_right)
        ticks_to_next_piece -= 0.05
        #print(ticks_to_next_piece)
        if ticks_to_next_piece < 5 :
            for piece in pieces:
                if sensor_data[0]==True:
                    take_screen_shot=True
                    print("Screenshot successfully saved")
                    pygame.image.save(screen,"screenshot"+str(i)+".png")
                    i=i+1
                sensor_1_measures.append(a1)
                sensor_2_measures.append(a2)                                
                space.remove(piece,piece.body)
                pieces.remove(piece)
                #print('piece removed')
        if ticks_to_next_piece <= 0:
            ticks_to_next_piece = 25
            piece_shape = add_piece(space)
            pieces.append(piece_shape)
            
        
        screen.fill((255,255,255))


        space.debug_draw(draw_options)
        space.step(1/50.0)
        
        
        tnr = pygame.font.SysFont('Arial', 20)
        text = tnr.render(str((a2,a1)), False, (0, 0 ,255 ))
        text2=tnr.render(str(sensor_data), False, (255, 0 ,0 ))
        screen.blit(text, (250,500))
        screen.blit(text2, (300,300))
        
        pygame.display.flip()
        take_screen_shot=False
        
        clock.tick(50)
        
if __name__ == '__main__':
    main()
    

Screenshot successfully saved


KeyboardInterrupt: 

In [ ]:
#501,200,0,200,space /// 43,200,600,200,space
#501,200,0,200

In [48]:
from math import radians
degrees(0.19)


10.886198107485642